In [1]:
import os
import pandas as pd
from blockbaxAnalyser import blockbaxAnalyser
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# --------------- SETTINGS -----------------------------------------
verbose = False # Set to True to get intermediate prints of results
plots = False
sb_numbers = [148098, 148097, 148091, 148088, 148076, 148096]
temp_steps = [-20, -10, 0, 10, 20, 30, 40, 50]
temp_steps_short = [-10, 0, 10, 20]
y_steps = [-10, -5, -2, 0, 2, 5, 10]
tl_min = -2 # min x value of trendline
tl_max = 2 # max x value of trendline

plot_from_date = "2022-04-28 18:30:00"
plot_till_date = "2022-05-05 01:00:00"

In [3]:
# --------------- INITIALISATION --------------------------------------
analyser = blockbaxAnalyser(sb_numbers = sb_numbers, 
plot_from_date=plot_from_date, 
plot_till_date=plot_till_date)

analyser.dataPrep(ref_filename=os.path.join(os.path.abspath('').removesuffix('blockbaxAnalyser.py'),"Ref","jewell_ref_april.txt"), 
offset_date_start="2022-05-02 03:40:00",
offset_date_end="2022-05-02 05:35:00")

if verbose:
    i=0
    print(analyser.dfs)
    print(sb_numbers[i])
    print(analyser.dfs[i].to_string())

In [4]:
# ------------------- GROUPING ---------------------------------------
bins_y = [-11, -7, -3.5, -1, 1, 3.5, 7, 10]
bins_temp = [-25, -15, -5, 5, 15, 25, 35, 45, 55]
for i in range(len(sb_numbers)):
    analyser.dfs[i]["Y setting"] = pd.cut(analyser.dfs[i]["Y value"], bins = bins_y, labels=y_steps)
    analyser.dfs[i]["Temp setting"] = pd.cut(analyser.dfs[i]["Temperature"], bins = bins_temp, labels=temp_steps)
    if verbose:
        print(analyser.dfs[i])
        print(analyser.dfs[0].to_string())


In [5]:
# ---------------- FUNCTION DEFINITIONS FOR STDTF -----------------------------
def get_corresponding_ymeans(y_setting):
    for y in range(len(y_steps)):
        if y_setting == y_steps[y]:
            return y_means[y]
def get_corresponding_ystds(y_setting):
    for y in range(len(y_steps)):
        if y_setting == y_steps[y]:
            return y_stds[y]

def get_corresponding_yrefmeans(y_setting):
    for y in range(len(y_steps)):
        if y_setting == y_steps[y]:
            return yref_means[y]
def get_corresponding_yrefstds(y_setting):
    for y in range(len(y_steps)):
        if y_setting == y_steps[y]:
            return yref_stds[y]

def get_corresponding_tempmeans(temp_setting):
    for t in range(len(temp_steps)):
        if temp_setting == temp_steps[t]:
            return temp_means[t]
def get_corresponding_tempstds(temp_setting):
    for t in range(len(temp_steps)):
        if temp_setting == temp_steps[t]:
            return temp_stds[t]
# ------------------- STANDARD TRANSFORM -------------------------------
y_lists = []
yref_lists = []
for y in y_steps:
    y_list = []
    yref_list = []
    for i in range(len(sb_numbers)):
        y_list += analyser.dfs[i].groupby("Y setting").get_group(y)["Y value"].tolist()
        yref_list += analyser.dfs[i].groupby("Y setting").get_group(y)["Reference"].tolist()
    y_lists.append(y_list)
    yref_lists.append(yref_list)
y_means = []
yref_means = []
y_stds = []
yref_stds = []
for y in y_lists:
    y_means.append(sum(y)/len(y))
    y_stds.append(np.std(y))
for yref in yref_lists:
    yref_means.append(sum(yref)/len(yref))
    yref_stds.append(np.std(yref))

temp_lists = []
for temp in temp_steps:
    temp_list = []
    for i in range(len(sb_numbers)):
            temp_list += analyser.dfs[i].groupby("Temp setting").get_group(temp)["Temperature"].tolist()
    temp_lists.append(temp_list)
temp_means = []
temp_stds = []
for temp in temp_lists:
    temp_means.append(sum(temp)/len(temp))
    temp_stds.append(np.std(temp))

for i in range(len(sb_numbers)): # Per smartbrick
    analyser.dfs[i]["Y mean"] = analyser.dfs[i].apply(lambda x: get_corresponding_ymeans(x["Y setting"]), axis=1)
    analyser.dfs[i]["Y std"] = analyser.dfs[i].apply(lambda x: get_corresponding_ystds(x["Y setting"]), axis=1)

    analyser.dfs[i]["Ref mean"] = analyser.dfs[i].apply(lambda x: get_corresponding_yrefmeans(x["Y setting"]), axis=1)
    analyser.dfs[i]["Ref std"] = analyser.dfs[i].apply(lambda x: get_corresponding_yrefstds(x["Y setting"]), axis=1)

    analyser.dfs[i]["Temp mean"] = analyser.dfs[i].apply(lambda x: get_corresponding_tempmeans(x["Temp setting"]), axis=1)
    analyser.dfs[i]["Temp std"] = analyser.dfs[i].apply(lambda x: get_corresponding_tempstds(x["Temp setting"]), axis=1)

    analyser.dfs[i]["Y stdtf"] = (analyser.dfs[i]["Y value"]-analyser.dfs[i]["Y mean"])/analyser.dfs[i]["Y std"]
    analyser.dfs[i]["Yref stdtf"] = (analyser.dfs[i]["Reference"]-analyser.dfs[i]["Ref mean"])/analyser.dfs[i]["Ref std"]
    analyser.dfs[i]["Temp stdtf"] = (analyser.dfs[i]["Temperature"]-analyser.dfs[i]["Temp mean"])/analyser.dfs[i]["Temp std"]

if verbose:
    print("Y means: ", y_means)
    print("Y standard deviations: ", y_stds)
    print("Temp means: ", temp_means)
    print("Temp standard deviations: ", temp_stds)
    print(analyser.dfs)
    print(analyser.dfs[0].to_string())

Hieronder is de data visueel uitgezet. Elk plotje is van één specifieke smartbrick (zie legenda voor serienummer) bij één specifieke hoek. Per smartbrick zijn er dus 7 plots. Dit omdat de sterke correlatie die eerder ontdekt was ook bij een test was waarbij we één hoek aanhielden en verschillende temperaturen testten.

In [10]:
# -------------------- PLOT STANDARD TRANSFORMED DATA (T, Y) ------------------------------
for i in range(len(sb_numbers)):
    for y in range(len(y_steps)):
        analyser.trendlineStdTF_per_y(i,y_value=y_steps[y], y_index=y)
        if plots:
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temp stdtf'], analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Y stdtf'], label=str(sb_numbers[i])+ ' , '+str(y_steps[y]))
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temp stdtf'], analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Yref stdtf'], label="Reference")
            if analyser.slope != None:
                x = np.linspace(-2,2,1000)
                y = analyser.slope[y] * x + analyser.intercept[y]
                plt.plot(x,y)

            plt.legend()
            plt.title("Standard transformed Temperature vs Y value")
            plt.xlabel('Temperature')
            plt.ylabel('Angle Y')

            plt.show() 
    analyser.slope = None
    analyser.intercept = None   

----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -10
slope: -1.052028, intercept: 0.324076
R-squared: 0.853671
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -5
slope: -1.677597, intercept: 0.321764
R-squared: 0.931424
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -2
slope: -1.691564, intercept: 0.240584
R-squared: 0.909746
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  0
slope: -1.044426, intercept: 0.060450
R-squared: 0.463254
--------------------------------------------------------------------

Op het oog vallen er een paar dingen op (*Vragen*):
1. Geen erg sterke correlatie tussen temperatuur en Y voor alle SmartBricks
2. Bij de ingesteld hoek van 10 graden loopt de genormaliseerde temperatuur sterk uiteen.
3. Er zijn verschillen met de referentie, maar de referentie vertoont ook correlatie met de temperatuur. Aangezien de referentie in principe temperatuursgecompenseerd is en we deze als absolute waarheid nemen, kunnen we concluderen dat dit attribueerbaar is aan de beweging van de testopstelling door temperatuursinvloed (dus niet door de invloed van temperatuur op de sensor).

*Antwoorden*
1. Verdeel in groepen gebaseerd op R2 en slope
    - Gem. positieve slope: 148076, 148096, 148088 
    - Gem. negatieve slope: 148098, 148091, 148097
    - Hoge R2 (gem. >0.8): 148098,148076
    - Meh R2 (rond 0.5): 148091, 148097, 148088, 148096 
2. Analyse hieronder wijst uit dat dit komt door hysterese bij -10 graden inclinatie en -20 graden temperatuur
3. klopt
Opmerking: Een verschil met de vorige analyse is dat in de vorige analyse met genormaliseerde data alléén de Y genormaliseerd was, niet de temperatuur. (dit is hieronder geprobeerd)

In [11]:
# ------------------------- NON-STDTF TEMPERATURE -------------------------------
for i in range(len(sb_numbers)):
    for y in range(len(y_steps)):
        analyser.trendlineStdTF_per_y_notf_temp(i,y_value=y_steps[y], y_index=y)
        if plots:
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temperature'], analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Y stdtf'], label=str(sb_numbers[i])+ ' , '+str(y_steps[y]))
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temperature'], analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Yref stdtf'], label='reference')

            if analyser.slope != None:
                x = np.linspace(-20,50,1000)
                y = analyser.slope[y] * x + analyser.intercept[y]
                plt.plot(x,y)

            plt.legend()
            plt.title("Standard transformed Temperature vs Y value")
            plt.xlabel('Temperature')
            plt.ylabel('Angle Y')

            plt.show() 
    analyser.slope = None
    analyser.intercept = None

----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -10
slope: -0.048555, intercept: 0.980123
R-squared: 0.984474
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -5
slope: -0.058664, intercept: 1.125438
R-squared: 0.991064
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -2
slope: -0.054655, intercept: 1.190514
R-squared: 0.984499
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  0
slope: -0.033154, intercept: 0.683424
R-squared: 0.455353
--------------------------------------------------------------------

Uit deze plots zien we een redelijk sterke correlatie tussen de temperatuur en standaard getransformeerde Y. 
Hieruit wordt ook duidelijk dat de grote spreiding die waargenomen was in het vorige block volledig attribueerbaar is aan de hysterese die plaatsvindt omdat de y stappen [10, 5, 2, 0, 2, -2, -5, -10, 0, 10] zijn, en dus dat de hoek 10 2x voorkomt. Wat opmerkelijk is is dat de hysterese bij de 0 hoek wel over de hele temperatuursrange aanwezig is, maar bij de 10 hoek alleen bij de temperatuur -20.
Ook is er bij de 10 graden hoek een groot verschil tussen het gedrag van de referentie en het gedrag van de SmartBricks, waar deze zich op andere temperaturen redelijk hetzelfde gedragen.
*Conclusies*
- Gebaseerd op de bovenstaande plots kunnen we geen groepen maken; Alle smartbricks hebben een relatief hoge R2 en een negatieve slope, waarbij de R2 ronduit slecht is bij hoek 10 (zoals hierboven beschreven attribueerbaar aan de hysterese bij temperatuur -20)
- Vergeleken met standaard getransformeerde temperatuur:

*Vragen*
- Gebruiken we standaard getransformeerde temperatuur of ongetransformeerde temperatuur?
- Moeten we met het model de error (= verschil tussen referentie en gemeten/getransformeerde hoek) of de hoek proberen te voorspellen?

*Antwoorden*
- Ongetransformeerde temperatuur geeft betere correlatie maar getransformeerde temperatuur geeft beter inzicht in het model, ik ga voor nu verder met getransformeerde temperatuur
- Het model moet de error voorspellen, zie hieronder voor analyse op basis van error


In [15]:
# -------------------- PLOT STANDARD TRANSFORMED DATA ERROR (T, Y) ------------------------------
plots=False
for i in range(len(sb_numbers)):
    analyser.dfs[i]["Y stdtf error"] = analyser.dfs[i]["Y stdtf"] - analyser.dfs[i]["Yref stdtf"]
    for y in range(len(y_steps)):
        analyser.trendlineStdTF_per_y_error(i,y_value=y_steps[y], y_index=y)
        if plots:
            plt.scatter(analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Temp stdtf'], 
            analyser.dfs[i].groupby("Y setting").get_group(y_steps[y])['Y stdtf error'], 
            label=str(sb_numbers[i])+ ' , '+str(y_steps[y]))
            if analyser.slope != None:
                x = np.linspace(-2,2,1000)
                y = analyser.slope[y] * x + analyser.intercept[y]
                plt.plot(x,y)

            plt.legend()
            plt.title("Standard transformed Temperature vs Y value")
            plt.xlabel('Temperature')
            plt.ylabel('Angle Y')

            plt.show() 
    analyser.slope = None
    analyser.intercept = None 

----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -10
slope: -0.125212, intercept: 0.175632
R-squared: 0.642788
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -5
slope: -0.474017, intercept: 0.188049
R-squared: 0.867532
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  -2
slope: -0.406797, intercept: 0.255842
R-squared: 0.708621
----------------------------------------------------------------------------------------------------
Trendline on standard transformed data of SmartBrick  148098 at angle  0
slope: -0.542691, intercept: 0.091105
R-squared: 0.811790
--------------------------------------------------------------------

De conclusie hieruit is dat er wel een verdeling gemaakt kan worden uit het error-gedrag van de smartbricks, maar dat deze niet direct correspondeert met wat er uit de analyse van ruwe data komt. Dit is een probleem omdat we de smartbricks dan niet in het veld kunnen classificeren (aangezien voor het bepalen van de error zonder a priori kennis een referentie nodig is).

In [ ]:
# --------------- NEW MODELS -------------------------------
# One model for positive correlation and one for negative correlation
sbs_positive = [148076, 148096, 148088]
sbs_negative = [148098, 148091, 148097]
groups = [sbs_positive, sbs_negative]
analysers = []

for n in range(len(groups)):
    analysers.append(blockbaxAnalyser(sb_numbers= groups[n],
    plot_from_date=plot_from_date, 
    plot_till_date=plot_till_date))

    analysers[n].dataPrep(ref_filename=os.path.join(os.path.abspath('').removesuffix('blockbaxAnalyser.py'),"Ref","jewell_ref_april.txt"), 
        offset_date_start="2022-05-02 03:40:00",
        offset_date_end="2022-05-02 05:35:00")
    for i in range(len(groups[n])):
        analysers[n].dfs[i]["Y setting"] = pd.cut(analysers[n].dfs[i]["Y value"], bins = bins_y, labels=y_steps)
        analysers[n].dfs[i]["Temp setting"] = pd.cut(analysers[n].dfs[i]["Temperature"], bins = bins_temp, labels=temp_steps)

    y_lists = []
    yref_lists = []
    for y in y_steps:
        y_list = []
        yref_list = []
        for i in range(len(groups[n])):
            y_list += analysers[n].dfs[i].groupby("Y setting").get_group(y)["Y value"].tolist()
            yref_list += analysers[n].dfs[i].groupby("Y setting").get_group(y)["Reference"].tolist()
        y_lists.append(y_list)
        yref_lists.append(yref_list)
    y_means = []
    yref_means = []
    y_stds = []
    yref_stds = []
    for y in y_lists:
        y_means.append(sum(y)/len(y))
        y_stds.append(np.std(y))
    for yref in yref_lists:
        yref_means.append(sum(yref)/len(yref))
        yref_stds.append(np.std(yref))

    temp_lists = []
    for temp in temp_steps:
        temp_list = []
        for i in range(len(groups[n])):
                temp_list += analysers[n].dfs[i].groupby("Temp setting").get_group(temp)["Temperature"].tolist()
        temp_lists.append(temp_list)
    temp_means = []
    temp_stds = []
    for temp in temp_lists:
        temp_means.append(sum(temp)/len(temp))
        temp_stds.append(np.std(temp))

    for i in range(len(sbs_positive)): # Per smartbrick
        pos.dfs[i]["Y mean"] = pos.dfs[i].apply(lambda x: get_corresponding_ymeans(x["Y setting"]), axis=1)
        pos.dfs[i]["Y std"] = pos.dfs[i].apply(lambda x: get_corresponding_ystds(x["Y setting"]), axis=1)

        pos.dfs[i]["Ref mean"] = pos.dfs[i].apply(lambda x: get_corresponding_yrefmeans(x["Y setting"]), axis=1)
        pos.dfs[i]["Ref std"] = pos.dfs[i].apply(lambda x: get_corresponding_yrefstds(x["Y setting"]), axis=1)

        pos.dfs[i]["Temp mean"] = pos.dfs[i].apply(lambda x: get_corresponding_tempmeans(x["Temp setting"]), axis=1)
        pos.dfs[i]["Temp std"] = pos.dfs[i].apply(lambda x: get_corresponding_tempstds(x["Temp setting"]), axis=1)

        pos.dfs[i]["Y stdtf"] = (pos.dfs[i]["Y value"]-pos.dfs[i]["Y mean"])/pos.dfs[i]["Y std"]
        pos.dfs[i]["Yref stdtf"] = (pos.dfs[i]["Reference"]-pos.dfs[i]["Ref mean"])/pos.dfs[i]["Ref std"]
        pos.dfs[i]["Temp stdtf"] = (pos.dfs[i]["Temperature"]-pos.dfs[i]["Temp mean"])/pos.dfs[i]["Temp std"]


To do list
[x] Group data based on y setting and temp setting
[x] Normalise data based on stdtransform
[x] Plot normalised data and determine R2 and trendline
[x] Analyse and evaluate
[x] Group smartbricks based on R2 and trendline slope/intersect
[ ] Create model for well-correlated smartbricks